#  IMDB All U.S. Released Movies: 1972-2016 Analysis

In [50]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Creating pandas empty data frame 
data will be filled after reading url

In [280]:
columns =['name','year','runtime', 'certificate','genre','rating','url']
mov_df = pd.DataFrame(columns=columns)

## Collecting Movie name, year, duration, certificate, genre, rating

In [282]:
from datetime import datetime
print(str(datetime.now()))
rows = []
for count in range(1,101):
    html = urlopen("http://www.imdb.com/list/ls057823854/?sort=list_order,asc&st_dt=&mode=detail&page="+str(count))
    bsObj = BeautifulSoup(html.read(), "lxml");
    nameList=bsObj.find_all("div", class_="lister-item-content")
    name=url=certificate=genre = 'None'
    year=runtime=rating=''
    for h in nameList:
        name = h.find('a').text
        if h.find('span',class_="lister-item-year text-muted unbold") is not None:
            year = h.find('span',class_="lister-item-year text-muted unbold").text
        if h.find('span',class_="runtime") is not None:
            runtime = h.find('span',class_="runtime").text
        if h.find('div',class_="inline-block ratings-imdb-rating") is not None:
            if 'data-value' in h.find('div',class_="inline-block ratings-imdb-rating").attrs.keys():
                rating = h.find('div',class_="inline-block ratings-imdb-rating").attrs['data-value']
        url = h.find('a').attrs['href']
        if h.find('span',class_="certificate") is not None:
            certificate = h.find('span',class_="certificate").text
        if h.find('span',class_="genre") is not None:
            genre = h.find('span',class_="genre").text
        row=[name,year,runtime,certificate,genre.strip(' \t\n\r'),rating,'http://www.imdb.com'+url.split("?",1)[0]]
        rows.append(row)
        
        
df = pd.DataFrame(rows, columns=columns)
   
print(str(datetime.now()))
print(df.head(5))

2017-12-23 23:21:13.046054
2017-12-23 23:32:18.596046
                                 name    year  runtime certificate  \
0                        Pulp Fiction  (1994)  154 min           R   
1            The Amazing Spider-Man 2  (2014)  142 min       PG-13   
2            The Shawshank Redemption  (1994)  142 min           R   
3  Star Wars: Episode IV - A New Hope  (1977)  121 min          PG   
4                  Back to the Future  (1985)  116 min          PG   

                        genre rating                                   url  
0                Crime, Drama    8.9  http://www.imdb.com/title/tt0110912/  
1   Action, Adventure, Sci-Fi    6.7  http://www.imdb.com/title/tt1872181/  
2                Crime, Drama    9.3  http://www.imdb.com/title/tt0111161/  
3  Action, Adventure, Fantasy    8.7  http://www.imdb.com/title/tt0076759/  
4   Adventure, Comedy, Sci-Fi    8.5  http://www.imdb.com/title/tt0088763/  


##  Persisting data

In [283]:
df.to_csv('IMDB_Movie_List.csv')

## Collecting Parental guide info for collected movies

* column for Movie Id so that we can join later

In [370]:
df['movie_id'] = df.url.str[-10:-1]
print(df.head())

                                 name    year  runtime certificate  \
0                        Pulp Fiction  (1994)  154 min           R   
1            The Amazing Spider-Man 2  (2014)  142 min       PG-13   
2            The Shawshank Redemption  (1994)  142 min           R   
3  Star Wars: Episode IV - A New Hope  (1977)  121 min          PG   
4                  Back to the Future  (1985)  116 min          PG   

                        genre rating                                   url  \
0                Crime, Drama    8.9  http://www.imdb.com/title/tt0110912/   
1   Action, Adventure, Sci-Fi    6.7  http://www.imdb.com/title/tt1872181/   
2                Crime, Drama    9.3  http://www.imdb.com/title/tt0111161/   
3  Action, Adventure, Fantasy    8.7  http://www.imdb.com/title/tt0076759/   
4   Adventure, Comedy, Sci-Fi    8.5  http://www.imdb.com/title/tt0088763/   

    movie_id  
0  tt0110912  
1  tt1872181  
2  tt0111161  
3  tt0076759  
4  tt0088763  


###  Creating 5x4 variable to store paraental guidance 
Each movies is reviewed under five catogeries
* Sex & Nudity
* Violence & Gore
* Profanity 
* Alcohol, Drugs & Smoking
* Frightening & Intense Scenes



Under Each Categoriy user has four levels to rate the movies
* None
* Mild
* Moderat
* Severe

In [360]:
p_columns =['movie_id','nudity','nudity_none','nudity_mild','nudity_moderate','nudity_severe',
        'violence','violence_none','violence_mild','violence_moderate','violence_severe',
        'profanity','profanity_none','profanity_mild','profanity_moderate','profanity_severe',
        'alcohol','alcohol_none','alcohol_mild','alcohol_moderate','alcohol_severe',
        'frightening','frightening_none','frightening_mild','frightening_moderate','frightening_severe']

In [367]:
couter=len(df)

#### Get Parents Guide data 

In [369]:
print(str(datetime.now()))
p_rows = []
for i in range(0,couter):
    movie_id = df['movie_id'][i]
    p_url=df['url'][i]+'parentalguide'
    p_html = urlopen(p_url)
    p_bsObj = BeautifulSoup(p_html.read(), "lxml")
    p_nameList=p_bsObj.find_all("section",class_='article listo content-advisories-index')
    nudity_none=nudity_mild=nudity_moderate=nudity_severe=0
    violence_none=violence_mild=violence_moderate=violence_severe=0
    profanity_none=profanity_mild=profanity_moderate=profanity_severe=0
    alcohol_none=alcohol_mild=alcohol_moderate=alcohol_severe=0
    frightening_none=frightening_mild=frightening_moderate=frightening_severe=0
    nudity=violence=profanity=alcohol=frightening=''
    for h in p_nameList:
        til = h.find_all('section')

        for p in til:
            
            A_None=Mild=Moderate=Severe=0
            if not (p.get('id') == 'certificates'  or 'spoiler' in p.get('id')) :
                advisory=p.find('h4').text
                if p.find('span',class_='advisory-severity-vote__vote-button-container') is not None:
                    strArr=p.find('span',class_='advisory-severity-vote__vote-button-container').text.splitlines()
                    str_list = list(filter(None, strArr))
                    none=str_list[1]
                    mild=str_list[3]
                    moderate=str_list[5]
                    severe=str_list[7]
                if p.find('div',class_='advisory-severity-vote__container ipl-zebra-list__item') is not None:
                    strArr=p.find('div',class_='advisory-severity-vote__container ipl-zebra-list__item').text.splitlines()
                    str_list = list(filter(None, strArr))
                    temp_advisory=str_list[0]
                
                if advisory =='Sex & Nudity':
                    nudity = temp_advisory
                    nudity_none=none
                    nudity_mild=mild
                    nudity_moderate=moderate
                    nudity_severe=severe
                    
                elif advisory == 'Violence & Gore':
                    violence = temp_advisory
                    violence_none=none
                    violence_mild=mild
                    violence_moderate=moderate
                    violence_severe=severe
                    
                elif advisory == 'Profanity':
                    profanity = temp_advisory
                    profanity_none=none
                    profanity_mild=mild
                    profanity_moderate = moderate
                    profanity_severe=severe
                elif advisory == 'Alcohol, Drugs & Smoking':
                    alcohol = temp_advisory
                    alcohol_none=none
                    alcohol_mild=mild
                    alcohol_moderate=moderate
                    alcohol_severe=severe
                elif advisory == 'Frightening & Intense Scenes':
                    frightening = temp_advisory
                    frightening_none=none
                    frightening_mild=mild
                    frightening_moderate=moderate
                    frightening_severe=severe
                
            
    row=(movie_id,nudity,nudity_none,nudity_mild,nudity_moderate,nudity_severe,
        violence,violence_none,violence_mild,violence_moderate,violence_severe,
        profanity,profanity_none,profanity_mild,profanity_moderate,profanity_severe,
        alcohol,alcohol_none,alcohol_mild,alcohol_moderate,alcohol_severe,
        frightening,frightening_none,frightening_mild,frightening_moderate,frightening_severe)
    p_rows.append(row)
    


p_df = pd.DataFrame(p_rows, columns=p_columns)
   
print(str(datetime.now()))
print(p_df.head(5))

p_df.to_csv('Movie_PG.csv')    




2017-12-24 02:52:29.779545
2017-12-24 04:03:43.137053
    movie_id    nudity nudity_none nudity_mild nudity_moderate nudity_severe  \
0  tt0110912  Moderate           0           4              16             5   
1  tt1872181      Mild           0           1               0             0   
2  tt0111161  Moderate           1           3               9             0   
3  tt0076759      None          13           1               0             0   
4  tt0088763      Mild           0           5               1             0   

   violence violence_none violence_mild violence_moderate        ...          \
0    Severe             0             0                 9        ...           
1  Moderate             0             0                 2        ...           
2  Moderate             0             2                 7        ...           
3      Mild             0             9                 5        ...           
4      Mild             0             4                 0        

## Merge both dataframe

In [627]:
final_df=pd.merge(df, p_df, on='movie_id', how='outer')
# final_df.to_csv('IMDB_Complete_data.csv')    
final_df.head(5)

,name,year,runtime,certificate,genre,rating,url,movie_id,nudity,nudity_none,...,alcohol,alcohol_none,alcohol_mild,alcohol_moderate,alcohol_severe,frightening,frightening_none,frightening_mild,frightening_moderate,frightening_severe
0,Pulp Fiction,(1994),154 min,R,"Crime, Drama",8.9,http://www.imdb.com/title/tt0110912/,tt0110912,Moderate,0,...,Severe,0,0,2,22,Severe,0,3,5,15
1,The Amazing Spider-Man 2,(2014),142 min,PG-13,"Action, Adventure, Sci-Fi",6.7,http://www.imdb.com/title/tt1872181/,tt1872181,Mild,0,...,Mild,0,2,0,0,Moderate,0,0,2,0
2,The Shawshank Redemption,(1994),142 min,R,"Crime, Drama",9.3,http://www.imdb.com/title/tt0111161/,tt0111161,Moderate,1,...,Mild,0,9,0,0,Moderate,1,2,5,1
3,Star Wars: Episode IV - A New Hope,(1977),121 min,PG,"Action, Adventure, Fantasy",8.7,http://www.imdb.com/title/tt0076759/,tt0076759,None,13,...,Mild,2,11,0,0,Mild,0,11,3,0
4,Back to the Future,(1985),116 min,PG,"Adventure, Comedy, Sci-Fi",8.5,http://www.imdb.com/title/tt0088763/,tt0088763,Mild,0,...,Mild,0,5,0,0,Mild,0,5,0,0


### Assign appropriate datatypes

In [628]:
import re
# temp_s=final_df.year.apply(lambda x: '' if x is None else str(x))

final_df['year']=final_df.year.apply(lambda x: str(re.findall('\d+', x)).strip("['']"))
final_df.year

0        1994
1        2014
2        1994
3        1977
4        1985
5        1985
6        1985
7        1991
8        1993
9        1994
10       1995
11       1995
12       1998
13       1998
14       1999
15       1999
16       1999
17       2000
18       2000
19       2001
20       2001
21       2002
22       2002
23       2003
24       2003
25       2004
26       2004
27       2005
28       2005
29       2005
         ... 
10059    2015
10060    2014
10061    2015
10062    2014
10063    2015
10064        
10065        
10066    2018
10067        
10068    2015
10069    2016
10070    2015
10071        
10072        
10073        
10074        
10075        
10076    2018
10077    2014
10078        
10079    2015
10080    2016
10081        
10082    2015
10083    2015
10084    2016
10085    2017
10086        
10087    2017
10088        
Name: year, dtype: object

In [630]:
final_df['year']=final_df[['year']].apply(pd.to_numeric,  errors='coerce')
final_df.year

0        1994.0
1        2014.0
2        1994.0
3        1977.0
4        1985.0
5        1985.0
6        1985.0
7        1991.0
8        1993.0
9        1994.0
10       1995.0
11       1995.0
12       1998.0
13       1998.0
14       1999.0
15       1999.0
16       1999.0
17       2000.0
18       2000.0
19       2001.0
20       2001.0
21       2002.0
22       2002.0
23       2003.0
24       2003.0
25       2004.0
26       2004.0
27       2005.0
28       2005.0
29       2005.0
          ...  
10059    2015.0
10060    2014.0
10061    2015.0
10062    2014.0
10063    2015.0
10064       NaN
10065       NaN
10066    2018.0
10067       NaN
10068    2015.0
10069    2016.0
10070    2015.0
10071       NaN
10072       NaN
10073       NaN
10074       NaN
10075       NaN
10076    2018.0
10077    2014.0
10078       NaN
10079    2015.0
10080    2016.0
10081       NaN
10082    2015.0
10083    2015.0
10084    2016.0
10085    2017.0
10086       NaN
10087    2017.0
10088       NaN
Name: year, dtype: float

In [435]:
# print(final_df.dtypes)

final_df['runtime']=final_df['runtime'].str.split(' ').str[0]
final_df[['rating', 'runtime']] = final_df[['rating', 'runtime']].apply(pd.to_numeric, errors='coerce')
final_df[['nudity_none','nudity_mild','nudity_moderate','nudity_severe',
        'violence_none','violence_mild','violence_moderate','violence_severe',
        'profanity_none','profanity_mild','profanity_moderate','profanity_severe',
        'alcohol_none','alcohol_mild','alcohol_moderate','alcohol_severe',
        'frightening_none','frightening_mild','frightening_moderate','frightening_severe']]= final_df[['nudity_none','nudity_mild','nudity_moderate','nudity_severe',
        'violence_none','violence_mild','violence_moderate','violence_severe',
        'profanity_none','profanity_mild','profanity_moderate','profanity_severe',
        'alcohol_none','alcohol_mild','alcohol_moderate','alcohol_severe',
        'frightening_none','frightening_mild','frightening_moderate','frightening_severe']].apply(pd.to_numeric, errors='coerce')
print('=====================================================================')
# print(final_df.dtypes)

name                    object
year                    object
runtime                 object
certificate             object
genre                   object
rating                  object
url                     object
movie_id                object
nudity                  object
nudity_none             object
nudity_mild             object
nudity_moderate         object
nudity_severe           object
violence                object
violence_none           object
violence_mild           object
violence_moderate       object
violence_severe         object
profanity               object
profanity_none          object
profanity_mild          object
profanity_moderate      object
profanity_severe        object
alcohol                 object
alcohol_none            object
alcohol_mild            object
alcohol_moderate        object
alcohol_severe          object
frightening             object
frightening_none        object
frightening_mild        object
frightening_moderate    object
frighten

#### Exploring  data
* Get different certificates and pick list for analysis

In [574]:
print(final_df.shape)

(10089, 33)


In [637]:
# As per web data there are less than 10089 row - find and remove dulplicate 
temp_df=final_df
print(temp_df[temp_df.duplicated(['movie_id'], keep=False)].shape)
temp_df=temp_df.drop_duplicates(subset=['movie_id'], keep=False)
print(temp_df.shape)

(169, 33)
(9920, 33)


In [638]:
temp_df.certificate.unique()

array(['R', 'PG-13', 'PG', 'G', 'Not Rated', 'TV-MA', 'NC-17', 'Unrated',
       'TV-PG', 'None', 'TV-14', 'X', 'Passed', 'TV-G', 'GP', 'Approved',
       'TV-Y', 'TV-Y7'], dtype=object)

#####  My aim is to find the correlation between MPAA film ratings and Parental Guidance reviews. I am picking only five certificate relevent to Movie certificate.
* R
* PG-13
* PG
* G
* NC

In [639]:
sel_cert=['R', 'PG-13', 'PG', 'G', 'NC-17']
temp_df = temp_df[temp_df['certificate'].isin(sel_cert)]
print(temp_df.shape)

(8542, 33)


In [640]:
temp_df.certificate.unique()

array(['R', 'PG-13', 'PG', 'G', 'NC-17'], dtype=object)

In [641]:
temp_df.to_csv('temp_df.csv')

In [642]:
temp_df.head()

,name,year,runtime,certificate,genre,rating,url,movie_id,nudity,nudity_none,...,alcohol,alcohol_none,alcohol_mild,alcohol_moderate,alcohol_severe,frightening,frightening_none,frightening_mild,frightening_moderate,frightening_severe
0,Pulp Fiction,1994.0,154 min,R,"Crime, Drama",8.9,http://www.imdb.com/title/tt0110912/,tt0110912,Moderate,0,...,Severe,0,0,2,22,Severe,0,3,5,15
1,The Amazing Spider-Man 2,2014.0,142 min,PG-13,"Action, Adventure, Sci-Fi",6.7,http://www.imdb.com/title/tt1872181/,tt1872181,Mild,0,...,Mild,0,2,0,0,Moderate,0,0,2,0
2,The Shawshank Redemption,1994.0,142 min,R,"Crime, Drama",9.3,http://www.imdb.com/title/tt0111161/,tt0111161,Moderate,1,...,Mild,0,9,0,0,Moderate,1,2,5,1
3,Star Wars: Episode IV - A New Hope,1977.0,121 min,PG,"Action, Adventure, Fantasy",8.7,http://www.imdb.com/title/tt0076759/,tt0076759,None,13,...,Mild,2,11,0,0,Mild,0,11,3,0
4,Back to the Future,1985.0,116 min,PG,"Adventure, Comedy, Sci-Fi",8.5,http://www.imdb.com/title/tt0088763/,tt0088763,Mild,0,...,Mild,0,5,0,0,Mild,0,5,0,0


## Next : Get Plot Keywords data and merge with above data

In [277]:
keywords_url=new_df['url'][0]+'keywords'
print(keywords_url)
 

http://www.imdb.com/title/tt0110912/keywords


In [438]:
keywords_html = urlopen(keywords_url)
keywords_bsObj = BeautifulSoup(keywords_html.read(), "lxml");